# Notebook for the creation of the disk lsh hashes for both datasets

Sheet that converts the extracted data from the data/chosen_data folder to hashes that will be stored in data/hashed_data/disk


In [3]:
# Importing nescessary modules
import os
import sys
import shutil
import timeit as ti
from tqdm import tqdm

from multiprocessing import Pool

def find_project_root(target_folder="masteroppgave"):
    """Find the absolute path of a folder by searching upward."""
    currentdir = os.path.abspath("__file__")  # Get absolute script path
    while True:
        if os.path.basename(currentdir) == target_folder:
            return currentdir  # Found the target folder
        parentdir = os.path.dirname(currentdir)
        if parentdir == currentdir:  # Stop at filesystem root
            return None
        currentdir = parentdir  # Move one level up

# Example usage
project_root = find_project_root("masteroppgave")

if project_root:
    sys.path.append(project_root)
    print(f"Project root found: {project_root}")
else:
    raise RuntimeError("Could not find 'masteroppgave' directory")

from utils.helpers.save_trajectory import save_trajectory_hashes
from utils.helpers import file_handler as fh
from utils.helpers import metafile_handler as mfh
from schemes.lsh_disk import DiskLSH

Project root found: c:\Users\eivin\dev\JoonEndreLSH\masteroppgave


In [4]:
# Declaring global variables:

from constants import (
    P_MAX_LON,
    P_MIN_LON,
    P_MAX_LAT,
    P_MIN_LAT,
    R_MAX_LON,
    R_MIN_LON,
    R_MAX_LAT,
    R_MIN_LAT,
)

SHOULD_DELETE_OLD_FILES = True


PORTO_OUTPUT_FOLDER = "../../dataset/hashed_data/disk/porto/"
ROME_OUTPUT_FOLDER = "../../dataset/hashed_data/disk/rome/"

PORTO_DATA_FOLDER = "../../dataset/porto/output/"
ROME_DATA_FOLDER = "../../dataset/rome/output/"

# Rome LSH Disk

Continuing with the rome set


In [18]:
# Run this cell to clear the chosen files in the ROME folder

if SHOULD_DELETE_OLD_FILES:
    fh.delete_old_files(ROME_OUTPUT_FOLDER, ".gitkeep")

In [15]:
# Creating a disk based LSH hashing object over rome

layers = 4
diameter = 1.5
num_disks = 50
meta_file = f"{ROME_OUTPUT_FOLDER}META-50.txt"

DiskRome = DiskLSH(
    "Rome D1",
    R_MIN_LAT,
    R_MAX_LAT,
    R_MIN_LON,
    R_MAX_LON,
    num_disks,
    layers,
    diameter,
    meta_file,
    ROME_DATA_FOLDER,
)

In [20]:
# Copying meta_files as well
meta_files = mfh.get_meta_files(ROME_DATA_FOLDER)

for filename in meta_files:
    shutil.copy(ROME_DATA_FOLDER + filename, ROME_OUTPUT_FOLDER)

# Generating the hashes and storing them in output folder along with the meta-files
hashes = DiskRome.compute_dataset_hashes_with_KD_tree()

save_trajectory_hashes(ROME_OUTPUT_FOLDER, hashes)

# Porto LSH Disk

Beginning with the porto set


In [24]:
# Run this cell to clear the chosen files in the PORTO folder

if SHOULD_DELETE_OLD_FILES:
    fh.delete_old_files(PORTO_OUTPUT_FOLDER, ".gitkeep")

In [25]:
# Create a disk-based LSH object over Porto

layers = 4
diameter = 1.5
num_disks = 50
meta_file = f"{PORTO_OUTPUT_FOLDER}META-100.txt"

DiskPorto = DiskLSH(
    "Porto D1",
    P_MIN_LAT,
    P_MAX_LAT,
    P_MIN_LON,
    P_MAX_LON,
    num_disks,
    layers,
    diameter,
    meta_file,
    PORTO_DATA_FOLDER,
)

In [26]:
# Copying meta_files as well
meta_files = mfh.get_meta_files(PORTO_DATA_FOLDER)

for filename in meta_files:
    shutil.copy(PORTO_DATA_FOLDER + filename, PORTO_OUTPUT_FOLDER)

# Generating the disk-based LSH objects hashes and saving them to file

hashes = DiskPorto.compute_dataset_hashes_with_KD_tree()

save_trajectory_hashes(PORTO_OUTPUT_FOLDER, hashes)

# Measuring run-times of hash generation

The cells below are created to measure the time-efficiency of the hash computation


In [27]:
# #Cell for measrung DiskLSH hash generation times for all methods in both datasets - using coordiante hashes
# import pandas as pd
# from itertools import chain

# output_folder = "schemes/experiments/runtimes/"
# file_name = "hashing_runtimes_disk_lsh.csv"

# hashing_map = {
#     "porto_naive" : hashing.fun_wrapper_p_naive,
#     "porto_quadrants" : hashing.fun_wrapper_p_quadrants,
#     "porto_kd_tree" : hashing.fun_wrapper_p_KD_tree,
#     "rome_naive" : hashing.fun_wrapper_r_naive,
#     "rome_quadrants" : hashing.fun_wrapper_r_quadrants,
#     "rome_kd_tree" : hashing.fun_wrapper_r_KD_tree
# }

# config = {
#     "porto" : [1000, 60, 4, 2.2],
#     # "rome" : [1000, 50, 5, 1.6]

# }

# runs = 10

# df = pd.DataFrame(columns=[f"Run_{run+1}" for run in range(runs)])

# for key in hashing_map.keys():
#     with Pool() as pool:
#         result = pool.map(hashing_map[key], [config[key.split("_")[0]] for _ in range(runs)])
#         df.loc[key] = list(chain.from_iterable(result))

# df.to_csv(os.path.join(output_folder, file_name))

NameError: name 'hashing' is not defined